In [ ]:
from importlib import reload
import ipynb.fs.full.notebook_viewer
reload(ipynb.fs.full.notebook_viewer)
from ipynb.fs.full.notebook_viewer import Viewer, root, bucket, fname2id, id2fname, split_dump, Filter
import pickle
from pickle import Unpickler
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
from vivid.utils import timer
import subprocess
from subprocess import PIPE
from tqdm.notebook import tqdm
from time import time
from functools import lru_cache
import numpy as np
from PIL import Image
import imagesize

In [ ]:
class TQDMBytesReader(object):
    def __init__(self, fd, **kwargs):
        self.fd = fd
        self.tqdm = tqdm(**kwargs)
    def read(self, size=-1):
        bytes = self.fd.read(size)
        self.tqdm.update(len(bytes))
        return bytes
    def readline(self):
        bytes = self.fd.readline()
        self.tqdm.update(len(bytes))
        return bytes
    def __enter__(self):
        self.tqdm.__enter__()
        return self
    def __exit__(self, *args, **kwargs):
        return self.tqdm.__exit__(*args, **kwargs)
def tqdm_load(fname):
    with open(fname, "rb") as fd:
         total = Path(fname).stat().st_size
         with TQDMBytesReader(fd, total=total) as pbfd:
             up = Unpickler(pbfd)
             obj = up.load()
    return obj

In [ ]:
readable_ori =  set((root/"danbooru2020/dump_readable_ori/0").read_text().strip().split())
missing2x = {"4005075","685111","3554125","3004206","1783239","528288","1717363","4268380","750472","1106509","2072514","2072517","3185545","4039613","3051636","4089762","800782","2515799","1195810","4260855"}

fire_eggs = dict()
for k in "duplicates is_deleted no_humans not-image photo text_only_page".split():
    fire_eggs[k] = set((root/f"danbooru2020/Danbooru2019/reduce/{k}.txt").read_text().strip().split())
    
ge512 =  set((root/"danbooru2020/ge512.ids").read_text().strip().split())
ge1024 =  set((root/"danbooru2020/ge1024.ids").read_text().strip().split())

i = 2
if not "ids" in globals():
    with timer(prefix="read_id_pkl"): # 311[s]
        ids = tqdm_load(root/f"danbooru2020/{i}_id.pkl")
if not "tags" in globals():
    with timer(prefix="read_tag_pkl"): # 58[s]
        tags = tqdm_load(root/f"danbooru2020/{i}_tag.pkl")

In [ ]:
@lru_cache(maxsize=None)
def t(s):
    if "|" in s:
        retval = reduce(set.__or__, map(tags.get, s.split("|")))
    else:
        retval = reduce(set.__and__, map(tags.get, s.split("&")))
    return retval

@lru_cache(maxsize=None)
def load_json(fname):
    fname = str(fname)
    retval = dotdict({k.replace("detection_", "").replace("bbox_", "").replace("class_", ""): np.array(v) for k, v in json.loads(Path(fname).read_text()).items()})
    retval["fname"] = fname
    retval["id"] = fname2id(fname)
    return retval

@lru_cache(maxsize=None)
def load_pkl(fname):
    fname = str(fname)
    with open(fname, "rb") as f:
        retval = dotdict(pickle.load(f))
    return retval

In [ ]:
def id2original(_id):
    return id2fname(_id, prefix="original", ext=lambda _id: ids[_id]["file_ext"])

def id2original2x(_id):
    if _id in tags["highres"]:
        return id2original(_id)
    return id2fname(_id, prefix="original2x", ext=lambda _: "png")

def id2original2xwhite(_id):
    if _id in tags["highres"]:
        return id2fname(_id, prefix="originalwhite", ext=lambda _: "png")
    return id2fname(_id, prefix="original2xwhite", ext=lambda _: "png")

def id2json(_id):
    return id2fname(_id, prefix="solo_json", bucket=lambda _: ".", ext=lambda _: "json")

def id2pkl(_id):
    return id2fname(_id, prefix="yaas/solov2out", ext=lambda _: "pkl")

def grouping(fnames, load=load_json, f=lambda x: np.where(x.label == 1)[0]):
    r = dotdict()
    r.fnames = tuple(map(str, fnames))
    r.zero = set()
    r.one = set()
    r.multi = set()
    pbar =  tqdm(r.fnames)
    for i, fname in enumerate(pbar):
        _id = fname2id(fname)
        x = load(fname)
        r[_id] = x
        fx = f(x)
        if len(fx) == 0:
            r.zero.add(_id)
        elif len(fx) == 1:
            r.one.add(_id)
        else:
            r.multi.add(_id)
        if i+1 == len(r.fnames):
            pbar.set_description(str(len(r.one)/(i+1)*100)[:2]+"%")
    return r

In [ ]:
solo = t("1girl|solo|1boy")-t("1girl&1boy")-t("2boys|2girls|3boys|3girls|4boys|4girls|5boys|5girls|6+boys|6+girls|multiple_boys|multiple_girls|multiple_views|variations")
bg = t("white_background|transparent_background")
#bg = t("white_background|transparent_background|simple_background")-t("black_background|dark_background")
exc0 = t('|'.join([
    "no_humans|arachne|pokemon_(creature)|octoling|monster_girl|isabelle_(animal_crossing)|animal_nose|animal_hug",
    "robot|cyborg|mecha_musume|machinery|doll_joints|crewmate_(among_us)|kamen_rider_faiz|olimar",
    "red_skin|blue_skin|purple_skin|wrinkled_skin|green_skin|two-tone_skin",
    "monochrome|lineart|text_only_page|photo-referenced|3d|unconventional_media|comic",
    "photo_(medium)|reference_photo_inset|reference_photo|photorealistic",
    "full_armor|weapon|chibi|cannon|mask|eyepatch",
]))
exc1 = t('|'.join([
    "zoom_layer",
    "back|ass|from_behind|from_above|bent_over|dutch_angle|standing_split",
    "car|bicycle",
    "translation_request",
    "fox_tail",
    "otter_costume",
    "sonic_the_hedgehog",
    "mystery_skulls",
    "floating_head",
]))|reduce(set.__or__, fire_eggs.values())

In [ ]:
fl = Filter()
fl["solo"]         = solo&readable_ori - missing2x
fl["exc0"]         = fl[-1]-exc0
fl["standing"]     = fl[-1]&tags["standing"]
fl["ge512"]        = fl[-1]&ge512
face     = grouping(map(id2json, fl["ge512"]), load=load_json, f=lambda x: np.where(x.label == 1)[0])
instance = grouping(map(id2pkl,  fl["ge512"]), load=load_pkl,  f=lambda x: x.pred_classes)
fl["faceinstance"] = fl[-1]&face.one&instance.one
fl["bg"]           = fl[-1]&bg
fl["censored"]     = fl[-1]-tags["censored"]
fl["safe"]         = {_id for _id in fl["censored"] if ids[_id]["rating"] == "s"}
fl["questionable"] = {_id for _id in fl["censored"] if ids[_id]["rating"] == "q"}
fl["explicit"]     = {_id for _id in fl["censored"] if ids[_id]["rating"] == "e"}
fl["full_body"]    = fl["safe"]&tags["full_body"]
fl["out"]          = fl["safe"]-exc1

In [ ]:
Viewer(map(id2original2x, fl["out"]), name="out_score", N=11, key = lambda _id: -int(ids[_id]["score"]))

In [ ]:
def convert_option(_id, dname, xc=0.5, yc=0.28, yw=0.235, size=512):
    fname = id2original(_id)
    j = load_json(id2json(_id))
    src = dotdict()
    dst = dotdict()
    src.X, src.Y = imagesize.get(fname)
    i = np.where(j.label==1)[0][0]
    src.Xmin = src.X*j.xmin[i]
    src.Xmax = src.X*j.xmax[i]
    src.Ymin = src.Y*j.ymin[i]
    src.Ymax = src.Y*j.ymax[i]
    src.Ywid = src.Ymax-src.Ymin
    src.Xcen = (src.Xmax+src.Xmin)/2
    src.Ycen = (src.Ymax+src.Ymin)/2
    dst = dotdict()
    dst.Xcen = size*xc
    dst.Ycen = size*yc
    dst.Ywid = size*yw
    z = dst.Ywid/src.Ywid
    option = [
        "-background", "white",
        "-resize", f"{round(100*z)}%",
        "-crop", f"{size}x{size}+{round(z*src.Xcen-dst.Xcen)}+{round(z*src.Ycen-dst.Ycen)}!",
        "-flatten",
        "-strip",
        "-quality", "100",
        f"{fname}",
        f"PNG24:{dname}",
    ]
    return option
options = list()
for _id in tqdm(out_score):
    option = " ".join(convert_option(_id, id2fname(_id, prefix="v3")))
    options.append(option)
split_dump(options, "option_v3")

In [ ]:
sample = list()
for fname in tqdm(list((root/"danbooru2020/v3").glob("**/*.png"))):
    sample.append( np.array(Image.open(fname)) )
Image.fromarray(np.mean(sample, axis=0).astype("uint8"))

In [ ]:
def th_option(fname, dname):
    return f"/home/natsuki/waifu2x/waifu2x.lua -m scale -i {fname} -o {dname}".split()
options = list()
for _id in tqdm(list((fl["ge512"]-tags["highres"])|face.zero)):
    option = " ".join(th_option(_id, id2fname(id2original(_id), prefix="original2x")))
    options.append(option)

In [ ]:
split_dump(options, "option_2x")

In [ ]:
options = list()
for fname in tqdm(list(map(str, (root/"getchu_s5").glob("*")))):
    dname = fname.replace("getchu_s5", "getchu_s5_2x")
    option = " ".join(th_option(fname, dname))
    options.append(option)

In [ ]:
split_dump(options, "option_getchu_s5_2x")

In [ ]:
for i in range(0, 75+1):
    for line in (root/f"danbooru2020/dump_option_2x/{i}").read_text().strip().split("\n"):
        fname = line.split()[-3]
        _id = fname2id(fname)
        tname = f"/tmp{_id}.png"
        if Path(dname).is_file():
            pass
        else:
            print(f"convert {fname} ")

In [ ]:
fnames = list(map(id2pkl, fl["ge512"]))
g = grouping(fnames, load=load_pkl, f=lambda x: x.pred_classes)

In [ ]:
for _id in instance.one:
    img = Image.fromarray((255*instance[_id].pred_masks[0])).convert("1")
    break
img
#h, w = map(round, np.array(np.where(p.pred_masks[0])).mean(axis=1))

In [ ]:
Image.fromarray((255*p.pred_masks[0]).astype("uint8")).convert("1")

In [ ]:
nbg = fl["faceinstance"]-t("simple_background")-white-exc1
print(len(nbg))

In [ ]:
options_convert = list()
options_mogrify = list()
for i, _id in enumerate(tqdm(instance.one)):
    srcname = id2original2xwhite(_id)
#    src = np.array(Image.open(fname))
#    str(root/f"danbooru2020/yaas/solov2out/{bucket(_id)}/{_id}.png")
#    str(root/f"danbooru2020/original2xwhitesolov2/{bucket(_id)}/{_id}.png")
    maskname  = id2fname(_id, prefix="yaas/solov2out", ext= lambda _:"png")
    dstname = id2fname(_id, prefix="original2xwhitesolov2", ext=lambda _:"png")
    options_convert.append( f"{srcname} {maskname} -compose CopyOpacity -composite {dstname}" )
    options_mogrify.append( f"-background white -flatten PNG24:{dstname}")
#    mask = instance[_id].pred_masks[0]
#    H, W = mask.shape
#    arr = 255*np.ones((H, W, 4), dtype="uint8")
#    arr[:,:,3] = 255*mask
#    img = Image.fromarray(arr)
#    img = Image.fromarray((255*mask).astype("uint8")).convert("1")
#    img.save(dname)
#    if i == 1000:
#        break

In [ ]:
split_dump(options_convert, "option_original2xwhitesolov2_convert")
split_dump(options_mogrify, "option_original2xwhitesolov2_mogrify")

In [ ]:
_id = "2832570"
mask = instance[_id].pred_masks[0]
#H, W = mask.shape
#arr = 255*np.ones((H, W, 4), dtype="uint8")
#arr[:,:,3] = 255*mask
#
img = Image.fromarray((255*mask).astype("uint8")).convert("1")
#img = Image.fromarray(arr)
img.save("/tmp/1.png")
img

In [ ]:
f"convert {fname} {dname} -compose CopyOpacity -composite /tmp/out.png"

In [ ]:
Viewer(map(id2fname, instance.multi))